In [ ]:
#!pip install chembl_webresource_client

In [1]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [2]:
target = new_client.target
target_query = target.search('coronavirus')
targets = pd.DataFrame.from_dict(target_query)

In [3]:
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Coronavirus,Coronavirus,17.0,False,CHEMBL613732,[],ORGANISM,11119
1,[],SARS coronavirus,SARS coronavirus,15.0,False,CHEMBL612575,[],ORGANISM,227859
2,[],Feline coronavirus,Feline coronavirus,15.0,False,CHEMBL612744,[],ORGANISM,12663
3,[],Human coronavirus 229E,Human coronavirus 229E,13.0,False,CHEMBL613837,[],ORGANISM,11137
4,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,10.0,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859
5,[],Middle East respiratory syndrome-related coron...,Middle East respiratory syndrome-related coron...,9.0,False,CHEMBL4296578,[],ORGANISM,1335626
6,"[{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...",SARS coronavirus,Replicase polyprotein 1ab,4.0,False,CHEMBL5118,"[{'accession': 'P0C6X7', 'component_descriptio...",SINGLE PROTEIN,227859
7,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,4.0,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049


### Download data for a selected target

In [88]:
def calc_label(row):
    if float(row['standard_value']) >= 10000.0:
        val = 'Inactive'
    elif float(row['standard_value']) <= 1000.0:
        val = 'Active'
    else:
        val = 'Intermediate'
    return val

def download_data_from_target(selected_target):
    activity = new_client.activity
    res = activity.filter(target_chembl_id = selected_target).filter(standard_type='IC50')
    df = pd.DataFrame.from_dict(res)

    df = df[df.standard_value.notnull()].copy()

    df = df[df.standard_value.notnull()]
    df = df[df.canonical_smiles.notnull()] 

    selection = ['molecule_chembl_id','canonical_smiles','standard_value'] 
    df = df[selection] 

    df['class'] = df.apply(calc_label, axis=1)

    df = df[df['class']!='Intermediate'].reset_index(drop=True)  
    
    filename = '../data/ChEMBL/bioactivity_data_preprocessed_'+ selected_target+'.csv'
    df.to_csv(filename, index=False)

    return df

In [89]:
selected_target = [targets.target_chembl_id[4], targets.target_chembl_id[6], targets.target_chembl_id[7]]
selected_target

['CHEMBL3927', 'CHEMBL5118', 'CHEMBL4523582']

In [94]:
#df1 = download_data_from_target(selected_target[1])

In [90]:
df0 = download_data_from_target(selected_target[0])
df1 = download_data_from_target(selected_target[1])
df2 = download_data_from_target(selected_target[2])

In [93]:
df0

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,13500.0,Inactive
1,CHEMBL426082,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,13110.0,Inactive
2,CHEMBL365134,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c(Br)cccc21,980.0,Active
3,CHEMBL190743,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccc(I)cc21,950.0,Active
4,CHEMBL365469,O=C1C(=O)N(Cc2cc3ccccc3s2)c2cccc(Cl)c21,11200.0,Inactive
...,...,...,...,...
114,CHEMBL2146517,COC(=O)[C@@]1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(C)c...,10600.0,Inactive
115,CHEMBL187460,C[C@H]1COC2=C1C(=O)C(=O)c1c2ccc2c1CCCC2(C)C,10100.0,Inactive
116,CHEMBL363535,Cc1coc2c1C(=O)C(=O)c1c-2ccc2c(C)cccc12,11500.0,Inactive
117,CHEMBL227075,Cc1cccc2c3c(ccc12)C1=C(C(=O)C3=O)[C@@H](C)CO1,10700.0,Inactive


In [99]:
df0.groupby('class').canonical_smiles.count()

class
Active       15
Inactive    104
Name: canonical_smiles, dtype: int64

In [100]:
df1.groupby('class').canonical_smiles.count()

class
Active        5
Inactive    156
Name: canonical_smiles, dtype: int64

In [101]:
df2.groupby('class').canonical_smiles.count()

class
Active      79
Inactive    34
Name: canonical_smiles, dtype: int64